### Installing all the required applications

In [23]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge gmaps --yes

print ('Geopy installed')
print ('Folium installed')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - gmaps


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geojson-2.4.1              |             py_0          15 KB  conda-forge
    gmaps-0.8.4                |             py_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following NEW packages will be INSTALLED:

  geojson            conda-forge/noarch::geojson-2.4.1-py_0
  gmaps              conda-forge/noarch::gmaps-0.8.4-py_0



geojson-2.4.1        | 15 KB     | ##################################### | 100% 
gmaps-0.8.4          | 1.9 MB    | ###################

### Importing all the required packages

In [2]:
import requests # library to handle requests 
import pandas as pd 
import numpy as np
import random


from geopy.geocoders import Nominatim

#libaries for displaying images 
from IPython.display import Image 
from IPython.core.display import HTML

#transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium
import lxml.html as lh

### URL for the Wikipedia page containing Toronto Neighbourhood data

In [3]:
url     = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Extracting the contents of the URL

In [4]:
page = requests.get(url)
#print (page.text)

### Storing the contents as doc parameter

In [5]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Extracting the table containing the Toronto Neighbourhoods from the URL content

In [6]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print '%d:"%s"'%(i,name)
    col.append((name,[]))

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]


[289, 289, 289]

### Creating a dataframe 'df' from the table extracted 

In [8]:
Dict={title:column for (title,column) in col }
df=pd.DataFrame(Dict)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df = df.replace('\n','', regex=True)
df.head()


,Neighbourhood,Postcode,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


In [9]:
df.columns

Index(['Neighbourhood\n', 'Postcode', 'Borough'], dtype='object')

### Removing the newline character from the 'Neighbourhood' column

In [10]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'}, inplace=True)
df.columns

Index(['Neighbourhood', 'Postcode', 'Borough'], dtype='object')

### Assigning Borough name to Neighbourhood if value of Neighbourhood = 'Not Assigned'

In [11]:
df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough
df.head()

,Neighbourhood,Postcode,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


### Extracting the Boroughs which are Not equal to 'Not Assigned'

In [12]:
df_filtered = df[df['Borough'] != 'Not assigned']
df_filtered.head()

,Neighbourhood,Postcode,Borough
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto
5,Regent Park,M5A,Downtown Toronto
6,Lawrence Heights,M6A,North York


### Grouping Neighbourhood for the same Postcode and Borough

In [13]:
df_Toranto = df_filtered.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_Toranto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Finding the shape of the Toronto Neighbourhood dataframe

In [14]:

df_Toranto.shape

(103, 3)

### Importing the coordinates of Toronto postcodes into Dataframe 

In [15]:
df_coords = pd.read_csv('./downloads/Geospatial_Coordinates.csv')
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Associting the location coordinates with the Borough Neighbourhoods

In [16]:
df_Totanto_coords = df_Toranto.set_index('Postcode').join(df_coords.set_index('Postal Code'))
df_Totanto_coords.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
df_Totanto_coords.reset_index(inplace=True)

In [18]:
df_Totanto_coords.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Filtering the Boroughs with value containing 'Toronto'

In [19]:
neighbourhoods = df_Totanto_coords[df_Totanto_coords['Borough'].str.contains("Toronto")]
neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Finding the coordinates of 'Toronto'

In [20]:

address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print ('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


### Plotting the Toronto Neighbourhoods on the map

In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label ='{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius =5,
    popup= label,
    color ='blue',
    fill = True,
    fill_color ='#3186cc',
    fill_opacity =0.7,
    parse_html=False).add_to(map_toronto)
    

folium.LayerControl().add_to(map_toronto)
map_toronto

In [22]:
def embed_map(m):
    from IPython.display import HTML

    m.save('index.html')
    with open('index.html') as f:
        html = f.read()

    iframe = '<iframe srcdoc="{srcdoc}" style="width: 100%; height: 750px; border: none"></iframe>'
    srcdoc = html.replace('"', '&quot;')
    return HTML(iframe.format(srcdoc=srcdoc))
embed_map(map_toronto)